In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import autogluon
from autogluon.tabular import TabularDataset,TabularPredictor
from sklearn.model_selection import train_test_split

c:\Users\km20186\project\signature\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))

C:\Users\km20186\AppData\Local\Temp\ipykernel_22660\3353662060.py:1: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
C:\Users\km20186\AppData\Local\Temp\ipykernel_22660\3353662060.py:2: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [3]:
train_df.columns

Index(['target_ym', 'money_room', 'building_id', 'building_status',
       'building_create_date', 'building_modify_date', 'building_type',
       'building_name', 'building_name_ruby', 'homes_building_name',
       ...
       'drugstore_distance', 'bank_distance', 'shopping_street_distance',
       'est_other_name', 'est_other_distance', 'statuses', 'parking_keiyaku',
       'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing'],
      dtype='object', length=149)

In [22]:
def data_preprocess(df):
    # 欠損値の割合が90%以上のカラムを削除
    missing_ratio = df.isnull().sum() / len(df)
    cols_to_drop = missing_ratio[missing_ratio >= 1].index.tolist()
    
    # 削除するカラムと欠損率を表示
    if cols_to_drop:
        print("削除するカラムと欠損率:")
        for col in cols_to_drop:
            print(f"  {col}: {missing_ratio[col]:.2%}")
    else:
        print("削除するカラムはありません")
    
    df = df.drop(columns=cols_to_drop)
    
    return df

In [23]:
df=data_preprocess(train_df)
df.columns

削除するカラムと欠損率:
  building_name_ruby: 100.00%
  name_ruby: 100.00%
  school_ele_code: 100.00%
  school_jun_code: 100.00%
  money_hoshou_company: 100.00%
  free_rent_duration: 100.00%
  free_rent_gen_timing: 100.00%


Index(['target_ym', 'money_room', 'building_id', 'building_status',
       'building_create_date', 'building_modify_date', 'building_type',
       'building_name', 'homes_building_name', 'homes_building_name_ruby',
       ...
       'super_distance', 'hospital_distance', 'park_distance',
       'drugstore_distance', 'bank_distance', 'shopping_street_distance',
       'est_other_name', 'est_other_distance', 'statuses', 'parking_keiyaku'],
      dtype='object', length=142)

In [4]:
train_data: pd.DataFrame = TabularDataset(os.path.join('..', 'data', 'train', 'train.csv'))
test_data: pd.DataFrame = TabularDataset(os.path.join('..', 'data', 'test', 'test.csv'))

In [7]:
train, test = train_test_split(train_df, test_size=0.2, random_state=42)
train_data: pd.DataFrame = TabularDataset(train)
test_data: pd.DataFrame = TabularDataset(test)
model=TabularPredictor(
    label='money_room',
    problem_type='regression',
    eval_metric='mape'
)
predictor=model.fit(
    train_data=train_data,
    test_data=test_data,
    presets='medium_quality',
    time_limit=360,
)

No path specified. Models will be saved in: "AutogluonModels\ag-20251204_022914"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       12.18 GB / 31.66 GB (38.5%)
Disk Space Avail:   77.30 GB / 237.73 GB (32.5%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 360s
AutoGluon will save models to "c:\Users\km20186\project\signature\my_code\AutogluonModels\ag-20251204_022914"
Train Data Rows:    291139
Train Data Columns: 148
Label Column:       money_room
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    13155.50 MB
	Train Data (Original)  Memory Usage: 103

[1000]	valid_set's l2: 3.4356e+13	valid_set's mean_absolute_percentage_error: -0.183779
[2000]	valid_set's l2: 3.03298e+13	valid_set's mean_absolute_percentage_error: -0.173058
[3000]	valid_set's l2: 2.85213e+13	valid_set's mean_absolute_percentage_error: -0.16814
[4000]	valid_set's l2: 2.75112e+13	valid_set's mean_absolute_percentage_error: -0.164961
[5000]	valid_set's l2: 2.67471e+13	valid_set's mean_absolute_percentage_error: -0.162503
[6000]	valid_set's l2: 2.61545e+13	valid_set's mean_absolute_percentage_error: -0.160379


	Ran out of time, early stopping on iteration 6722. Best iteration is:
	[6711]	valid_set's l2: 2.57546e+13	valid_set's mean_absolute_percentage_error: -0.159078
	-0.1591	 = Validation score   (-mean_absolute_percentage_error)
	273.58s	 = Training   runtime
	0.41s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 272.38s of the -2.64s of remaining time.
	Ensemble Weights: {'LightGBMXT': 1.0}
	-0.1591	 = Validation score   (-mean_absolute_percentage_error)
	0.0s	 = Training   runtime
	0.02s	 = Validation runtime
AutoGluon training complete, total runtime = 363.31s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 6765.3 rows/s (2912 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("c:\Users\km20186\project\signature\my_code\AutogluonModels\ag-20251204_022914")


In [8]:
predictions = predictor.predict(test)
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.161872,-0.159078,mean_absolute_percentage_error,9.885666,0.414435,273.583377,9.885666,0.414435,273.583377,1,True,1
1,WeightedEnsemble_L2,-0.161872,-0.159078,mean_absolute_percentage_error,9.917920,0.430431,273.583377,0.032254,0.015996,0.000000,2,True,2


In [11]:
predictor.feature_importance(test,
                             time_limit=3600
                            )

These features in provided data are not utilized by the predictor and will be ignored: ['building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing']
Computing feature importance via permutation shuffling for 141 features using 5000 rows with 10 shuffle sets... Time limit: 3600s...
	2520.61s	= Expected runtime (252.06s per shuffle set)
	2532.79s	= Actual runtime (Completed 10 of 10 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
house_area,2.300771e-01,0.006885,1.543239e-15,10,0.237152,0.223002
rosen_name1,1.865778e-01,0.005413,1.168899e-15,10,0.192141,0.181015
year_built,1.731723e-01,0.004244,2.564161e-16,10,0.177534,0.168810
post1,6.859550e-02,0.003063,5.635803e-14,10,0.071743,0.065448
rosen_name2,5.081582e-02,0.002925,5.516655e-13,10,0.053822,0.047810
...,...,...,...,...,...,...
reform_etc,0.000000e+00,0.000000,5.000000e-01,10,0.000000,0.000000
reform_common_area,-1.206994e-07,0.000002,5.654032e-01,10,0.000002,-0.000002
reform_common_area_date,-5.666912e-06,0.000012,9.195891e-01,10,0.000006,-0.000018
reform_place_other,-5.917251e-06,0.000011,9.416210e-01,10,0.000005,-0.000017
